# 🔬 Unsupervised Learning — Clustering (DBSCAN Focus)

**Course:** CSC582 — King Saud University 
**Reference:** Introduction to Machine Learning with Python — Chapter 3 (pp. 168–207) 
**GitHub:** [ClusteringInDBSCAN](https://github.com/YOUR_USERNAME/ClusteringInDBSCAN)

---

## Notebook Outline
1. **k-Means Clustering** — basics & failure cases
2. **Agglomerative Clustering** — hierarchical approach & dendrograms
3. **DBSCAN** ⭐ — density-based clustering (our focus)
4. **Comparing & Evaluating** — ARI, Silhouette Score
5. **Real-World Demo** — Iris dataset with all three algorithms

## Setup & Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import make_blobs, make_moons, load_iris
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.cluster import adjusted_rand_score, silhouette_score
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, ward

# Plot style
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print('All imports successful!')

---
## 1. k-Means Clustering (pp. 168–181)

**How it works:**
1. Choose k (number of clusters)
2. Randomly initialize k cluster centers
3. Assign each point to the nearest center
4. Recompute centers as the mean of assigned points
5. Repeat 3–4 until convergence

### 1.1 Basic k-Means

In [ ]:
# Generate synthetic 2D data with 3 blobs
X, y_true = make_blobs(n_samples=300, centers=3, random_state=1, cluster_std=0.60)

# Apply k-Means
kmeans = KMeans(n_clusters=3, random_state=0, n_init=10)
kmeans.fit(X)

print(f'Cluster labels (first 20): {kmeans.labels_[:20]}')
print(f'Cluster centers:\n{kmeans.cluster_centers_}')
print(f'Inertia (sum of squared distances): {kmeans.inertia_:.2f}')

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(X[:, 0], X[:, 1], c=y_true, cmap='viridis', s=40, alpha=0.7)
axes[0].set_title('Original Data (True Labels)', fontsize=14)
axes[0].set_xlabel('Feature 0'); axes[0].set_ylabel('Feature 1')

axes[1].scatter(X[:, 0], X[:, 1], c=kmeans.labels_, cmap='viridis', s=40, alpha=0.7)
axes[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
                c='red', marker='^', s=200, edgecolors='black', linewidth=2,
                label='Cluster Centers')
axes[1].set_title('k-Means Clustering (k=3)', fontsize=14)
axes[1].set_xlabel('Feature 0'); axes[1].set_ylabel('Feature 1')
axes[1].legend()

plt.tight_layout()
plt.show()

### 1.2 Elbow Method — Choosing Optimal k *(Enhancement)*

In [ ]:
inertias = []
sil_scores = []
K_range = range(2, 11)

for k in K_range:
    km = KMeans(n_clusters=k, random_state=0, n_init=10)
    km.fit(X)
    inertias.append(km.inertia_)
    sil_scores.append(silhouette_score(X, km.labels_))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (k)'); axes[0].set_ylabel('Inertia')
axes[0].set_title('Elbow Method', fontsize=14)
axes[0].axvline(x=3, color='red', linestyle='--', alpha=0.7, label='Optimal k=3')
axes[0].legend()

axes[1].plot(K_range, sil_scores, 'go-', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (k)'); axes[1].set_ylabel('Silhouette Score')
axes[1].set_title('Silhouette Score vs k', fontsize=14)
axes[1].axvline(x=3, color='red', linestyle='--', alpha=0.7, label='Optimal k=3')
axes[1].legend()

plt.tight_layout()
plt.show()
print('The elbow at k=3 confirms the optimal number of clusters.')

### 1.3 k-Means Failure Cases

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Case 1: Different densities
X_varied, y_varied = make_blobs(n_samples=200, cluster_std=[1.0, 2.5, 0.5], random_state=170)
y_pred = KMeans(n_clusters=3, random_state=0, n_init=10).fit_predict(X_varied)
axes[0].scatter(X_varied[:, 0], X_varied[:, 1], c=y_pred, cmap='viridis', s=40)
axes[0].set_title('Failure: Different Densities', fontsize=13)

# Case 2: Non-spherical / elongated clusters
X_blob, y_blob = make_blobs(random_state=170, n_samples=600)
rng = np.random.RandomState(74)
transformation = rng.normal(size=(2, 2))
X_aniso = np.dot(X_blob, transformation)
y_pred2 = KMeans(n_clusters=3, random_state=0, n_init=10).fit_predict(X_aniso)
axes[1].scatter(X_aniso[:, 0], X_aniso[:, 1], c=y_pred2, cmap='viridis', s=40)
axes[1].set_title('Failure: Non-Spherical Clusters', fontsize=13)

# Case 3: Two moons (complex shapes)
X_moons, y_moons = make_moons(n_samples=200, noise=0.05, random_state=0)
y_pred3 = KMeans(n_clusters=2, random_state=0, n_init=10).fit_predict(X_moons)
axes[2].scatter(X_moons[:, 0], X_moons[:, 1], c=y_pred3, cmap='viridis', s=40)
axes[2].set_title('Failure: Complex Shapes (Two Moons)', fontsize=13)

plt.suptitle('k-Means Failure Cases', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

---
## 2. Agglomerative Clustering (pp. 182–187)

**How it works (bottom-up):**
1. Each point starts as its own cluster
2. Find the two most similar (closest) clusters
3. Merge them into one
4. Repeat until desired number of clusters reached

**Linkage criteria:** Ward (default), Average, Complete

### 2.1 Basic Agglomerative Clustering

In [ ]:
X_agg, y_agg = make_blobs(random_state=1)

agg = AgglomerativeClustering(n_clusters=3)
assignment = agg.fit_predict(X_agg)

plt.figure(figsize=(8, 6))
plt.scatter(X_agg[:, 0], X_agg[:, 1], c=assignment, cmap='viridis', s=60,
            edgecolors='black', linewidth=0.5)
plt.xlabel('Feature 0'); plt.ylabel('Feature 1')
plt.title('Agglomerative Clustering (Ward Linkage, 3 Clusters)', fontsize=14)
plt.show()

### 2.2 Dendrogram

In [ ]:
X_dendro, y_dendro = make_blobs(random_state=0, n_samples=12)
linkage_array = ward(X_dendro)

plt.figure(figsize=(12, 6))
dendrogram(linkage_array)

ax = plt.gca()
bounds = ax.get_xbound()
ax.plot(bounds, [7.25, 7.25], '--', c='red', linewidth=2)
ax.plot(bounds, [4, 4], '--', c='blue', linewidth=2)
ax.text(bounds[1], 7.25, '  2 clusters', va='center', fontsize=13, color='red')
ax.text(bounds[1], 4, '  3 clusters', va='center', fontsize=13, color='blue')

plt.xlabel('Sample Index'); plt.ylabel('Cluster Distance (Ward)')
plt.title('Dendrogram — Hierarchical Clustering', fontsize=14)
plt.show()

---
## ⭐ 3. DBSCAN — Density-Based Clustering (pp. 187–190)

**Key advantages over k-Means and Agglomerative:**
- Does NOT require specifying number of clusters
- Handles complex, non-convex shapes
- Detects noise/outliers automatically

**Two parameters:**
- `eps` — radius of neighborhood around each point
- `min_samples` — minimum neighbors to be a core point

**Three point types:**
- **Core** — ≥ min_samples neighbors within eps (heart of cluster)
- **Border** — < min_samples neighbors, but within eps of a core point (edge of cluster)
- **Noise** — not near any core point → label = -1

### 3.1 DBSCAN Parameter Exploration

In [ ]:
X_db, y_db = make_blobs(random_state=0, n_samples=12)

print('Effect of eps and min_samples on clustering:')
print(f'{"min_samples":>12} {"eps":>6} {"clusters":>40}')
print('-' * 62)

for min_s in [2, 3, 5]:
    for eps_val in [1.0, 1.5, 2.0, 3.0]:
        db = DBSCAN(min_samples=min_s, eps=eps_val)
        clusters = db.fit_predict(X_db)
        print(f'{min_s:>12} {eps_val:>6.1f} {str(clusters):>40}')

### 3.2 Effect of `eps` Parameter

In [ ]:
X_moons, y_moons = make_moons(n_samples=300, noise=0.06, random_state=0)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_moons)

fig, axes = plt.subplots(2, 3, figsize=(18, 11))
eps_values = [0.1, 0.2, 0.3, 0.5, 0.8, 1.5]

for ax, eps_val in zip(axes.ravel(), eps_values):
    db = DBSCAN(eps=eps_val, min_samples=5)
    labels = db.fit_predict(X_scaled)
    
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)
    
    # Noise in red
    noise_mask = labels == -1
    ax.scatter(X_scaled[noise_mask, 0], X_scaled[noise_mask, 1],
               c='red', marker='x', s=50, label=f'Noise ({n_noise} pts)', zorder=3)
    # Cluster points
    cluster_mask = labels != -1
    if cluster_mask.any():
        ax.scatter(X_scaled[cluster_mask, 0], X_scaled[cluster_mask, 1],
                   c=labels[cluster_mask], cmap='viridis', s=40,
                   edgecolors='black', linewidth=0.3)
    
    # Show eps radius circle
    if eps_val <= 0.5:
        circle = plt.Circle((X_scaled[150, 0], X_scaled[150, 1]),
                           eps_val, fill=False, color='red', linewidth=2,
                           linestyle='--', alpha=0.7)
        ax.add_patch(circle)
        ax.plot(X_scaled[150, 0], X_scaled[150, 1], 'r*', markersize=15, zorder=5)
    
    if n_clusters == 0: result, color = 'ALL NOISE!', 'red'
    elif n_clusters == 2: result, color = 'CORRECT!', 'green'
    elif n_clusters == 1: result, color = 'One big cluster', 'orange'
    else: result, color = 'Too fragmented', 'orange'
    
    ax.set_title(f'eps = {eps_val}\n{n_clusters} clusters, {n_noise} noise — {result}',
                fontsize=13, fontweight='bold', color=color)
    ax.set_xlabel('Feature 0'); ax.set_ylabel('Feature 1')
    ax.legend(loc='upper right', fontsize=9)

fig.suptitle('EFFECT OF eps (min_samples fixed at 5)\n'
             'Red dashed circle = eps neighborhood radius',
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### 3.3 Effect of `min_samples` Parameter

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 11))
min_samples_values = [2, 3, 5, 10, 20, 50]

for ax, min_s in zip(axes.ravel(), min_samples_values):
    db = DBSCAN(eps=0.5, min_samples=min_s)
    labels = db.fit_predict(X_scaled)
    
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)
    
    # Identify point types
    core_mask = np.zeros(len(labels), dtype=bool)
    if hasattr(db, 'core_sample_indices_') and len(db.core_sample_indices_) > 0:
        core_mask[db.core_sample_indices_] = True
    
    noise_mask = labels == -1
    border_mask = (labels != -1) & (~core_mask)
    
    # Plot noise
    ax.scatter(X_scaled[noise_mask, 0], X_scaled[noise_mask, 1],
               c='red', marker='x', s=50, label=f'Noise ({n_noise})', zorder=3)
    # Plot border (small)
    if border_mask.any():
        ax.scatter(X_scaled[border_mask, 0], X_scaled[border_mask, 1],
                   c=labels[border_mask], cmap='viridis', s=30,
                   edgecolors='black', linewidth=0.3, alpha=0.6)
    # Plot core (large)
    if core_mask.any():
        ax.scatter(X_scaled[core_mask, 0], X_scaled[core_mask, 1],
                   c=labels[core_mask], cmap='viridis', s=60,
                   edgecolors='black', linewidth=0.5)
    
    n_core = core_mask.sum()
    n_border = border_mask.sum()
    ax.set_title(f'min_samples = {min_s}\n{n_clusters} clusters | '
                f'Core: {n_core} | Border: {n_border} | Noise: {n_noise}',
                fontsize=12, fontweight='bold')
    ax.set_xlabel('Feature 0'); ax.set_ylabel('Feature 1')
    ax.legend(loc='upper right', fontsize=9)

fig.suptitle('EFFECT OF min_samples (eps fixed at 0.5)\n'
             'Large dots = Core | Small dots = Border | Red X = Noise',
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### 3.4 DBSCAN vs k-Means vs Agglomerative on Two Moons

In [ ]:
X_moons, y_moons = make_moons(n_samples=200, noise=0.05, random_state=0)
scaler = StandardScaler()
X_moons_scaled = scaler.fit_transform(X_moons)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# k-Means (fails)
labels_km = KMeans(n_clusters=2, random_state=0, n_init=10).fit_predict(X_moons_scaled)
axes[0].scatter(X_moons_scaled[:, 0], X_moons_scaled[:, 1], c=labels_km, cmap='viridis', s=60)
axes[0].set_title('k-Means (k=2)\n❌ Fails on complex shapes!', fontsize=13)

# Agglomerative (fails)
labels_agg = AgglomerativeClustering(n_clusters=2).fit_predict(X_moons_scaled)
axes[1].scatter(X_moons_scaled[:, 0], X_moons_scaled[:, 1], c=labels_agg, cmap='viridis', s=60)
axes[1].set_title('Agglomerative (k=2)\n❌ Also fails!', fontsize=13)

# DBSCAN (succeeds!)
labels_db = DBSCAN(eps=0.5, min_samples=5).fit_predict(X_moons_scaled)
axes[2].scatter(X_moons_scaled[:, 0], X_moons_scaled[:, 1], c=labels_db, cmap='viridis', s=60)
axes[2].set_title('DBSCAN (eps=0.5)\n✅ Correctly separates!', fontsize=13)

for ax in axes:
    ax.set_xlabel('Feature 0'); ax.set_ylabel('Feature 1')

plt.suptitle('Two Moons: DBSCAN Succeeds Where Others Fail', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

### 3.5 DBSCAN Core / Border / Noise Visualization

In [ ]:
# Create data with visible noise points
X_demo, y_demo = make_moons(n_samples=200, noise=0.12, random_state=42)
scaler_demo = StandardScaler()
X_demo_scaled = scaler_demo.fit_transform(X_demo)

db_demo = DBSCAN(eps=0.4, min_samples=5)
labels_demo = db_demo.fit_predict(X_demo_scaled)

core_mask = np.zeros(len(labels_demo), dtype=bool)
core_mask[db_demo.core_sample_indices_] = True
noise_mask = labels_demo == -1
border_mask = (~core_mask) & (~noise_mask)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))

# Core points (large circles)
ax.scatter(X_demo_scaled[core_mask, 0], X_demo_scaled[core_mask, 1],
           c=labels_demo[core_mask], cmap='viridis', s=100,
           edgecolors='black', linewidth=1,
           label=f'Core Points ({core_mask.sum()})', zorder=3)

# Border points (small squares)
ax.scatter(X_demo_scaled[border_mask, 0], X_demo_scaled[border_mask, 1],
           c=labels_demo[border_mask], cmap='viridis', s=50,
           edgecolors='gray', linewidth=1, marker='s',
           label=f'Border Points ({border_mask.sum()})', zorder=2)

# Noise points (red X)
ax.scatter(X_demo_scaled[noise_mask, 0], X_demo_scaled[noise_mask, 1],
           c='red', s=80, marker='X', linewidth=1,
           label=f'Noise Points ({noise_mask.sum()})', zorder=4)

# Draw eps circles around two core points
for idx in db_demo.core_sample_indices_[:2]:
    circle = plt.Circle((X_demo_scaled[idx, 0], X_demo_scaled[idx, 1]),
                       0.4, fill=False, color='blue', linewidth=1.5,
                       linestyle='--', alpha=0.5)
    ax.add_patch(circle)

ax.set_xlabel('Feature 0', fontsize=13); ax.set_ylabel('Feature 1', fontsize=13)
ax.set_title('DBSCAN Point Types (eps=0.4, min_samples=5)\n'
             'Blue dashed circles = eps neighborhood',
             fontsize=14, fontweight='bold')
ax.legend(fontsize=12, loc='upper right')
plt.tight_layout()
plt.show()

---
## 4. Comparing & Evaluating Clustering (pp. 191–207)

### 4.1 Adjusted Rand Index (ARI) — With Ground Truth

In [ ]:
X_eval, y_eval = make_moons(n_samples=200, noise=0.05, random_state=0)
scaler = StandardScaler()
X_eval_scaled = scaler.fit_transform(X_eval)

random_state = np.random.RandomState(seed=0)
random_clusters = random_state.randint(low=0, high=2, size=len(X_eval))

algorithms = {
    'Random': random_clusters,
    'k-Means': KMeans(n_clusters=2, random_state=0, n_init=10).fit_predict(X_eval_scaled),
    'Agglomerative': AgglomerativeClustering(n_clusters=2).fit_predict(X_eval_scaled),
    'DBSCAN': DBSCAN().fit_predict(X_eval_scaled),
}

fig, axes = plt.subplots(1, 4, figsize=(20, 4))

print('ARI Scores (1.0 = perfect, 0.0 = random):')
for ax, (name, labels) in zip(axes, algorithms.items()):
    ari = adjusted_rand_score(y_eval, labels)
    print(f'  {name:20s}: {ari:.2f}')
    ax.scatter(X_eval_scaled[:, 0], X_eval_scaled[:, 1], c=labels, cmap='viridis', s=40)
    ax.set_title(f'{name}\nARI: {ari:.2f}', fontsize=12)

plt.suptitle('Adjusted Rand Index (ARI) Comparison', fontsize=15, y=1.05)
plt.tight_layout()
plt.show()

### 4.2 Why accuracy_score is WRONG for Clustering

In [ ]:
clusters1 = [0, 0, 1, 1, 0]
clusters2 = [1, 1, 0, 0, 1]  # Same grouping, labels just swapped!

print(f'Clusters1: {clusters1}')
print(f'Clusters2: {clusters2}  (identical grouping, different labels)')
print(f'\nAccuracy:  {accuracy_score(clusters1, clusters2):.2f}  <-- WRONG! Says 0%')
print(f'ARI:       {adjusted_rand_score(clusters1, clusters2):.2f}  <-- CORRECT! Says 100%')
print('\nLesson: Cluster labels are arbitrary. Always use ARI or NMI, never accuracy!')

### 4.3 Silhouette Score — Without Ground Truth

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 4))

print('Silhouette Scores (higher = more compact clusters):')
for ax, (name, labels) in zip(axes, algorithms.items()):
    n_unique = len(set(labels)) - (1 if -1 in labels else 0)
    sil = silhouette_score(X_eval_scaled, labels) if n_unique >= 2 else -1
    print(f'  {name:20s}: {sil:.2f}')
    ax.scatter(X_eval_scaled[:, 0], X_eval_scaled[:, 1], c=labels, cmap='viridis', s=40)
    ax.set_title(f'{name}\nSilhouette: {sil:.2f}', fontsize=12)

plt.suptitle('Silhouette Score Comparison', fontsize=15, y=1.05)
plt.tight_layout()
plt.show()
print('\nNote: k-Means scores HIGHER than DBSCAN even though DBSCAN is visually correct!')
print('Silhouette favors compact spherical clusters — it can be misleading.')

---
## 5. Real-World Demo — Iris Dataset *(Enhancement)*

In [ ]:
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

print(f'Iris dataset: {X_iris.shape[0]} samples, {X_iris.shape[1]} features')
print(f'Features: {iris.feature_names}')
print(f'True classes: {iris.target_names}')

# Scale and reduce to 2D for visualization
scaler_iris = StandardScaler()
X_iris_scaled = scaler_iris.fit_transform(X_iris)
pca_iris = PCA(n_components=2)
X_iris_2d = pca_iris.fit_transform(X_iris_scaled)

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# True labels
axes[0, 0].scatter(X_iris_2d[:, 0], X_iris_2d[:, 1], c=y_iris, cmap='viridis', s=50,
                    edgecolors='black', linewidth=0.5)
axes[0, 0].set_title('True Labels', fontsize=14)

# k-Means
labels_km = KMeans(n_clusters=3, random_state=0, n_init=10).fit_predict(X_iris_scaled)
ari_km = adjusted_rand_score(y_iris, labels_km)
axes[0, 1].scatter(X_iris_2d[:, 0], X_iris_2d[:, 1], c=labels_km, cmap='viridis', s=50,
                    edgecolors='black', linewidth=0.5)
axes[0, 1].set_title(f'k-Means (k=3) — ARI: {ari_km:.2f}', fontsize=14)

# Agglomerative
labels_agg = AgglomerativeClustering(n_clusters=3).fit_predict(X_iris_scaled)
ari_agg = adjusted_rand_score(y_iris, labels_agg)
axes[1, 0].scatter(X_iris_2d[:, 0], X_iris_2d[:, 1], c=labels_agg, cmap='viridis', s=50,
                    edgecolors='black', linewidth=0.5)
axes[1, 0].set_title(f'Agglomerative — ARI: {ari_agg:.2f}', fontsize=14)

# DBSCAN
labels_db = DBSCAN(eps=0.9, min_samples=5).fit_predict(X_iris_scaled)
ari_db = adjusted_rand_score(y_iris, labels_db)
n_noise = list(labels_db).count(-1)
axes[1, 1].scatter(X_iris_2d[:, 0], X_iris_2d[:, 1], c=labels_db, cmap='viridis', s=50,
                    edgecolors='black', linewidth=0.5)
axes[1, 1].set_title(f'DBSCAN — ARI: {ari_db:.2f} ({n_noise} noise pts)', fontsize=14)

for ax in axes.ravel():
    ax.set_xlabel('PCA Component 1'); ax.set_ylabel('PCA Component 2')

plt.suptitle('Clustering Algorithms on Iris Dataset', fontsize=16, y=1.01)
plt.tight_layout()
plt.show()

---
## 6. Summary — Algorithm Comparison

| Feature | k-Means | Agglomerative | DBSCAN |
|---------|---------|---------------|--------|
| Must set # clusters | Yes | Yes | **No** |
| Complex shapes | No | No | **Yes** |
| Noise detection | No | No | **Yes** |
| Scalability | Excellent | Good | Good |
| Cluster sizes | Even | Even | Varies |
| Interpretability | Cluster centers | Dendrogram | Core/border/noise |
| Key parameters | n_clusters | n_clusters, linkage | eps, min_samples |

---
## 7. Book's Real-World Example — Labeled Faces in the Wild (pp. 195–207)

This section replicates the book's analysis of the **Labeled Faces in the Wild (LFW)** dataset.
It demonstrates how each algorithm behaves on real, high-dimensional image data.

**Dataset:** 2,063 face images, each 87×65 pixels, reduced to 100 dimensions via PCA.

**Key findings from the book:**
- DBSCAN reveals outlier faces (odd angles, hands covering faces, wearing hats)
- k-Means produces smooth "average face" cluster centers
- Agglomerative clustering with 40 clusters finds meaningful face groups

### 7.1 Load and Prepare the Faces Dataset

In [ ]:
from sklearn.datasets import fetch_lfw_people

# Load faces — this may take a minute to download the first time
people = fetch_lfw_people(min_faces_per_person=20, resize=0.7)
image_shape = people.images[0].shape

print(f'Dataset shape: {people.images.shape}')
print(f'Image size: {image_shape}')
print(f'Number of people: {len(people.target_names)}')

# Limit to 50 images per person (to balance the dataset)
mask = np.zeros(people.target.shape, dtype=bool)
for target in np.unique(people.target):
    mask[np.where(people.target == target)[0][:50]] = 1

X_people = people.data[mask]
y_people = people.target[mask]

# Scale pixel values to 0-1
X_people = X_people / 255.

print(f'\nAfter balancing: {X_people.shape[0]} images')
print(f'Features per image: {X_people.shape[1]} (pixels)')

### 7.2 Sample Faces from the Dataset

In [ ]:
# Show some sample faces
fig, axes = plt.subplots(2, 5, figsize=(15, 8),
                         subplot_kw={'xticks': (), 'yticks': ()})
fig.suptitle('Sample Faces from the Dataset', fontsize=16, fontweight='bold')
for i, (image, label, ax) in enumerate(zip(X_people, y_people, axes.ravel())):
    ax.imshow(image.reshape(image_shape), cmap='gray')
    ax.set_title(people.target_names[label].split()[-1], fontsize=11)
plt.tight_layout()
plt.show()

### 7.3 PCA Preprocessing

Working with raw pixels (3,000+ features) is impractical.
We use PCA to reduce to 100 meaningful components — this is what the book does on p.195.

In [ ]:
# Reduce dimensions with PCA (as the book does)
pca = PCA(n_components=100, whiten=True, random_state=0)
pca.fit(X_people)
X_pca = pca.transform(X_people)

print(f'Original shape: {X_people.shape}')
print(f'After PCA:      {X_pca.shape}')
print(f'Variance explained: {pca.explained_variance_ratio_.sum():.1%}')

### 7.4 ⭐ DBSCAN on Faces — Outlier Detection (Book pp. 195–199)

The book shows how DBSCAN behaves very differently from the other algorithms:
- Default parameters → everything is noise
- With tuned eps → one big cluster + noise (outlier faces)
- The "noise" faces are genuinely unusual: hats, odd angles, hands in front of faces

In [ ]:
# Step 1: Default DBSCAN — everything is noise!
print('=== DBSCAN Tuning Process ===')
print()

dbscan_default = DBSCAN()
labels_default = dbscan_default.fit_predict(X_pca)
print(f'Default (eps=0.5): Unique labels = {np.unique(labels_default)}')
print('  → All points are noise! eps is too small for 100D data.')

# Step 2: Try min_samples=3
dbscan_ms3 = DBSCAN(min_samples=3)
labels_ms3 = dbscan_ms3.fit_predict(X_pca)
print(f'\nmin_samples=3 (eps=0.5): Unique labels = {np.unique(labels_ms3)}')
print('  → Still all noise. eps is the real problem.')

# Step 3: Increase eps
print('\n--- Exploring different eps values ---')
for eps in [1, 3, 5, 7, 9, 11, 13, 15]:
    dbscan = DBSCAN(eps=eps, min_samples=3)
    labels = dbscan.fit_predict(X_pca)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)
    sizes = np.bincount(labels + 1)  # +1 because noise is -1
    print(f'  eps={eps:>2}: {n_clusters} clusters, {n_noise} noise | sizes: {sizes}')

In [ ]:
# eps=15 gives 1 big cluster + 27 noise points — let's see the noise faces!
dbscan_15 = DBSCAN(min_samples=3, eps=15)
labels_15 = dbscan_15.fit_predict(X_pca)

noise_mask = labels_15 == -1
n_noise = noise_mask.sum()
print(f'eps=15: {n_noise} noise points (outlier faces)')
print(f'These are the "unusual" faces that DBSCAN flagged as not fitting any group.\n')

# Show the noise (outlier) faces
noise_images = X_people[noise_mask]
n_show = min(n_noise, 15)
cols = min(n_show, 5)
rows = (n_show + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(3 * cols, 3.5 * rows),
                         subplot_kw={'xticks': (), 'yticks': ()})
fig.suptitle('DBSCAN Noise Points — Outlier Faces (eps=15)',
             fontsize=15, fontweight='bold', color='red')

for i, ax in enumerate(axes.ravel()):
    if i < n_show:
        ax.imshow(noise_images[i].reshape(image_shape), cmap='gray')
        ax.set_title(f'Outlier {i+1}', fontsize=10, color='red')
    else:
        ax.set_visible(False)

plt.tight_layout()
plt.show()
print('Notice: odd angles, hats, hands in front of faces, unusual crops.')
print('This is OUTLIER DETECTION — a unique strength of DBSCAN!')

### 7.5 DBSCAN eps=7 — Finding Small Similar Clusters (Book p. 198)

In [ ]:
# eps=7 produces the most interesting result: many small clusters
dbscan_7 = DBSCAN(min_samples=3, eps=7)
labels_7 = dbscan_7.fit_predict(X_pca)

n_clusters = max(labels_7) + 1
n_noise = list(labels_7).count(-1)
print(f'eps=7: {n_clusters} clusters, {n_noise} noise points\n')

# Show each small cluster
for cluster in range(min(n_clusters, 10)):  # Show up to 10 clusters
    mask = labels_7 == cluster
    n_images = np.sum(mask)
    if n_images == 0:
        continue
    
    n_show = min(n_images, 8)
    fig, axes = plt.subplots(1, n_show + 1, figsize=(2.5 * (n_show + 1), 3),
                             subplot_kw={'xticks': (), 'yticks': ()})
    
    # Label
    axes[0].text(0.5, 0.5, f'Cluster {cluster}\n({n_images} faces)',
                ha='center', va='center', fontsize=12, fontweight='bold',
                transform=axes[0].transAxes)
    axes[0].set_frame_on(False)
    
    images = X_people[mask]
    labels_true = y_people[mask]
    for i, ax in enumerate(axes[1:]):
        if i < n_show:
            ax.imshow(images[i].reshape(image_shape), cmap='gray')
            ax.set_title(people.target_names[labels_true[i]].split()[-1], fontsize=9)
    
    plt.tight_layout()
    plt.show()

print('\nNotice: each cluster contains faces with similar orientation,')
print('expression, or the same person. DBSCAN finds genuine visual similarity!')

### 7.6 k-Means on Faces — Cluster Centers as "Average Faces" (Book pp. 200–202)

In [ ]:
# k-Means with 10 clusters
km_faces = KMeans(n_clusters=10, random_state=0, n_init=10)
labels_km_faces = km_faces.fit_predict(X_pca)

print(f'k-Means cluster sizes: {np.bincount(labels_km_faces)}')

# Show cluster centers as "average faces"
fig, axes = plt.subplots(2, 5, figsize=(15, 7),
                         subplot_kw={'xticks': (), 'yticks': ()})
fig.suptitle('k-Means Cluster Centers — "Average Faces" (k=10)',
             fontsize=15, fontweight='bold')

for i, (center, ax) in enumerate(zip(km_faces.cluster_centers_, axes.ravel())):
    # Transform cluster center back to image space
    face = pca.inverse_transform(center)
    ax.imshow(face.reshape(image_shape), cmap='gray', vmin=0, vmax=1)
    size = np.sum(labels_km_faces == i)
    ax.set_title(f'Cluster {i} ({size} faces)', fontsize=11)

plt.tight_layout()
plt.show()
print('Each center is a smooth "average" of all faces in that cluster.')
print('Notice different orientations, expressions, and lighting conditions.')

In [ ]:
# For each cluster: show center + 5 closest + 5 furthest faces
print('For each cluster: Center → 5 closest faces → 5 furthest faces')
print('(Closest = most typical, Furthest = least typical)\n')

for cluster_idx in range(10):
    mask = labels_km_faces == cluster_idx
    cluster_points = X_pca[mask]
    cluster_images = X_people[mask]
    
    # Distances to cluster center
    center = km_faces.cluster_centers_[cluster_idx]
    dists = np.sqrt(np.sum((cluster_points - center) ** 2, axis=1))
    
    closest = np.argsort(dists)[:5]
    furthest = np.argsort(dists)[-5:]
    
    fig, axes = plt.subplots(1, 11, figsize=(22, 2.5),
                             subplot_kw={'xticks': (), 'yticks': ()})
    
    # Center
    axes[0].imshow(pca.inverse_transform(center).reshape(image_shape),
                   cmap='gray', vmin=0, vmax=1)
    axes[0].set_title(f'Center {cluster_idx}', fontsize=9, color='blue', fontweight='bold')
    
    # 5 closest
    for i, idx in enumerate(closest):
        axes[1 + i].imshow(cluster_images[idx].reshape(image_shape), cmap='gray')
        axes[1 + i].set_title('closest', fontsize=8, color='green')
    
    # 5 furthest
    for i, idx in enumerate(furthest):
        axes[6 + i].imshow(cluster_images[idx].reshape(image_shape), cmap='gray')
        axes[6 + i].set_title('furthest', fontsize=8, color='red')
    
    plt.tight_layout()
    plt.show()

### 7.7 Agglomerative Clustering on Faces — Dendrogram (Book pp. 203–207)

In [ ]:
# Agglomerative with 10 clusters
agg_faces = AgglomerativeClustering(n_clusters=10)
labels_agg_faces = agg_faces.fit_predict(X_pca)

print(f'Agglomerative cluster sizes: {np.bincount(labels_agg_faces)}')
print(f'\nCompare with k-Means sizes: {np.bincount(labels_km_faces)}')

# ARI between k-Means and Agglomerative
ari_km_agg = adjusted_rand_score(labels_km_faces, labels_agg_faces)
print(f'\nARI between k-Means and Agglomerative: {ari_km_agg:.2f}')
print('Low ARI means they found quite different groupings!')

In [ ]:
# Dendrogram of the faces dataset
from scipy.cluster.hierarchy import dendrogram, ward

linkage_array = ward(X_pca)

plt.figure(figsize=(20, 6))
dendrogram(linkage_array, p=7, truncate_mode='level', no_labels=True)
plt.xlabel('Sample Index', fontsize=12)
plt.ylabel('Cluster Distance', fontsize=12)
plt.title('Dendrogram of Faces Dataset — No clear natural number of clusters',
          fontsize=15, fontweight='bold')
plt.show()
print('Unlike the toy datasets, there is no obvious "cut point" here.')
print('This confirms DBSCAN\'s finding: faces don\'t form distinct groups.')

In [ ]:
# Show sample faces from each agglomerative cluster
print('Sample faces from each Agglomerative cluster (10 clusters):\n')

for cluster in range(10):
    mask = labels_agg_faces == cluster
    n_images = np.sum(mask)
    n_show = min(n_images, 8)
    
    fig, axes = plt.subplots(1, n_show + 1, figsize=(2.5 * (n_show + 1), 3),
                             subplot_kw={'xticks': (), 'yticks': ()})
    
    axes[0].text(0.5, 0.5, f'Cluster {cluster}\n({n_images} faces)',
                ha='center', va='center', fontsize=12, fontweight='bold',
                transform=axes[0].transAxes)
    axes[0].set_frame_on(False)
    
    images = X_people[mask]
    labels_true = y_people[mask]
    for i, ax in enumerate(axes[1:]):
        if i < n_show:
            ax.imshow(images[i].reshape(image_shape), cmap='gray')
            ax.set_title(people.target_names[labels_true[i]].split()[-1], fontsize=9)
    
    plt.tight_layout()
    plt.show()

### 7.8 Final Comparison — All Three Algorithms on Faces

In [ ]:
print('=' * 65)
print('SUMMARY: Clustering Algorithms on Faces Dataset')
print('=' * 65)
print()
print(f'{"Algorithm":<25} {"Clusters":<12} {"Noise":<10} {"Key Finding"}')
print('-' * 65)
print(f'{"DBSCAN (eps=15)":<25} {"1":<12} {noise_mask.sum():<10} Outlier detection — unusual faces flagged')
print(f'{"DBSCAN (eps=7)":<25} {n_clusters:<12} {list(labels_7).count(-1):<10} Small clusters of very similar faces')
print(f'{"k-Means (k=10)":<25} {"10":<12} {"0":<10} Smooth average-face centers')
print(f'{"Agglomerative (k=10)":<25} {"10":<12} {"0":<10} Uneven sizes, different from k-Means')
print()
print('Key insight: Face images don\'t form distinct natural groups.')
print('DBSCAN revealed this honestly (1 big cluster + outliers),')
print('while k-Means and Agglomerative forced even partitions.')